In [ ]:
%%bash
mkdir -p /kaggle/working/submission

In [ ]:
# %%writefile submission/main.py
# %%writefile -a submission/main.py

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import os
import sys
import shutil

# torch.backends.cuda.enable_mem_efficient_sdp(False)
# torch.backends.cuda.enable_flash_sdp(False)

from utils import generate_answer
from utils_guesser import guess
from utils_asker import ask
from utils_answerer import answer

KAGGLE_AGENT_PATH = "/kaggle_simulations/agent/"
if os.path.exists(KAGGLE_AGENT_PATH):
    model_id = os.path.join(KAGGLE_AGENT_PATH, "1")
else:
    model_id = "/kaggle/input/llama-3/transformers/8b-chat-hf/1"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map="auto")
id_eot = tokenizer.convert_tokens_to_ids(["<|eot_id|>"])[0]


class Robot:
    def __init__(self):
        pass

    def on(self, mode, obs):
        try:
            import pickle
            pickle.dump(obs, open(f"obs_{mode}.pkl", "wb"))
        except:
            pass
        if mode == "asking":
            output = ask(obs, tokenizer, model, id_eot)
        elif mode == "answering":
            output = answer(obs, tokenizer, model, id_eot)
            if "yes" in output.lower():
                output = "yes"
            elif "no" in output.lower():
                output = "no"
            if ("yes" not in output.lower() and "no" not in output.lower()):
                output = "yes"
        elif mode == "guessing":
            output = guess(obs, tokenizer, model, id_eot)
        else:
            raise ValueError("mode can only take one of these values: asking, answering, guessing")
        return output


robot = Robot()


def agent(obs, cfg):
    if obs.turnType == "ask":
        response = robot.on(mode="asking", obs=obs)
    elif obs.turnType == "guess":
        response = robot.on(mode="guessing", obs=obs)
    elif obs.turnType == "answer":
        response = robot.on(mode="answering", obs=obs)
    else:
        raise ValueError("turnType can only take one of these values: ask, guess, answer")
    if response == None or len(response) <= 1:
        response = "yes"
    return response

In [ ]:
# %%time

# from kaggle_environments import make
# env = make("llm_20_questions", debug=True)
# game_output = env.run(agents=[agent, agent, agent, agent])

In [ ]:
# env.render(mode="ipython", width=600, height=500)

In [ ]:
!apt install pigz pv > /dev/null

In [ ]:
!tar --use-compress-program='pigz --fast --recursive | pv' -cf submission.tar.gz -C /kaggle/input/llama-3/transformers/8b-chat-hf . -C /kaggle/working/submission .

In [ ]:
# to see what's inside tar.gz file

# import tarfile
# tar = tarfile.open("/kaggle/working/submission.tar.gz")
# for file in tar.getmembers():
#     print(file.name)